In [1]:
import json
import requests
from urllib.parse import quote_plus, unquote_plus
from requests.utils import requote_uri
from warcio.archiveiterator import ArchiveIterator
from bs4 import BeautifulSoup

In [2]:
indices = {
    'CC-MAIN-2024-10', 'CC-MAIN-2024-18', 'CC-MAIN-2024-22',
    'CC-MAIN-2024-26', 'CC-MAIN-2024-30', 'CC-MAIN-2024-33',
    'CC-MAIN-2024-38', 'CC-MAIN-2024-42', 'CC-MAIN-2024-46'
}
search_query = 'ru.wikipedia.org/*'
keywords = [
    'Пермь', 'Пермский', 'Пермский Национальный Исследовательский Политехнический Университет', 'ПНИПУ',
    'ИТАС', 'МГУ', 'Московский государственный университет', 
    'МФТИ', 'Физтех', 'Пастернак', 'Борис Пастернак'
]

In [3]:
def search_url(index_name, target_url):
    encoded = quote_plus(target_url)
    query_url = f'http://index.commoncrawl.org/{index_name}-index?url={encoded}&output=json'
    response = requests.get(query_url)
    if response.status_code == 200:
        lines = response.text.strip().split('\n')
        return [json.loads(line) for line in lines]
    return None
def get_unique_records(records):
    unique_records = {}
    for record in records:
        url = record.get('url')
        if url and url not in unique_records:
            unique_records[url] = record
    return unique_records

In [4]:
all_results = []
for index in indices:
    found = search_url(index, search_query)
    if found:
        all_results.extend(found)
print(f"Всего найдено результатов: {len(all_results)}")
unique = get_unique_records(all_results)
print(f"Уникальных записей: {len(unique)}")

Всего найдено результатов: 122071
Уникальных записей: 118477


In [5]:
def fetch_record_content(filename, offset, length):
    s3_path = f'https://data.commoncrawl.org/{filename}'
    headers = {'Range': f'bytes={offset}-{offset + length - 1}'}
    resp = requests.get(s3_path, headers=headers, stream=True)
    if resp.status_code == 206:
        for record in ArchiveIterator(resp.raw):
            if record.rec_type == 'response':
                return record.content_stream().read()
    else:
        print(f"Ошибка при загрузке данных: {resp.status_code}")
    return None

In [6]:
filtered = [
    rec for rec in unique.values()
    if any(requote_uri(kw) in rec['url'] for kw in keywords)
]
print(f"Отфильтрованных результатов: {len(filtered)}")

Отфильтрованных результатов: 18


In [7]:
html_data = {}
for entry in filtered:
    content = fetch_record_content(entry['filename'], int(entry['offset']), int(entry['length']))
    if content:
        html_data[entry['url']] = content
for link, html_content in html_data.items():
    soup = BeautifulSoup(html_content, 'html.parser')
    title = soup.title.string if soup.title else 'Без названия'
    readable_url = unquote_plus(link)
    print(f'Найдена статья: {title}; По ссылке: {readable_url}')
    text = soup.get_text().lower()
    found_keywords = [word for word in keywords if word.lower() in text]
    if found_keywords:
        print(f"Найдено по ключевому слову: {', '.join(found_keywords)}")
    print("\n")

Найдена статья: Географический факультет МГУ — Википедия; По ссылке: https://ru.wikipedia.org/wiki/Географический_факультет_МГУ
Найдено по ключевому слову: МГУ, Московский государственный университет


Найдена статья: Геологический факультет МГУ — Википедия; По ссылке: https://ru.wikipedia.org/wiki/Геологический_факультет_МГУ
Найдено по ключевому слову: МГУ, Московский государственный университет


Найдена статья: Главное здание МГУ — Википедия; По ссылке: https://ru.wikipedia.org/wiki/Главное_здание_МГУ
Найдено по ключевому слову: МГУ, Московский государственный университет


Найдена статья: Категория:Выпускники исторического факультета МГУ — Википедия; По ссылке: https://ru.wikipedia.org/wiki/Категория:Выпускники_исторического_факультета_МГУ
Найдено по ключевому слову: МГУ


Найдена статья: Категория:Заслуженные профессора МГУ — Википедия; По ссылке: https://ru.wikipedia.org/wiki/Категория:Заслуженные_профессора_МГУ
Найдено по ключевому слову: МГУ


Найдена статья: Александровск (Пер